In [33]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import TimestampType, DateType
from pyspark.sql.functions import udf, col, monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek, to_timestamp

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#     spark = SparkSession \
#         .builder \
#         .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
#         .getOrCreate()
#     return spark

In [3]:
# config = configparser.ConfigParser()
# config.read('dl.cfg')

# os.environ['AWS_ACCESS_KEY_ID']=config['AWS_ACCESS_KEY_ID']
# os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS_SECRET_ACCESS_KEY']

In [14]:
print(os.path.join("s3a://udacity-dend/", "song_data/A/A/A/*.json"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3a://udacity-dend/song_data/A/A/A/*.json

In [15]:
#get filepath to song data file
song_data = os.path.join("s3a://udacity-dend/", "song_data/A/A/*/*.json")

# read song data to file
song_data_df = spark.read.json(song_data, mode='PERMISSIVE', columnNameOfCorruptRecord='corrupt_record')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
song_data_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [8]:
song_data_df.show(3)

+------------------+---------------+---------------+----------------+--------------------+----------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|artist_location|artist_longitude|         artist_name|  duration|num_songs|           song_id|               title|year|
+------------------+---------------+---------------+----------------+--------------------+----------+---------+------------------+--------------------+----+
|ARSUVLW12454A4C8B8|       35.83073|      Tennessee|       -85.97874|Royal Philharmoni...|  94.56281|        1|SOBTCUI12A8AE48B70|Faust: Ballet Mus...|   0|
|ARXQC081187FB4AD42|       54.31407|             UK|        -2.23001|William Shatner_ ...|1047.71873|        1|SOXRPUH12AB017F769|Exodus: Part I: M...|   0|
|ARWUNH81187FB4A3E0|           null|Miami , Florida|            null|         Trick Daddy| 227.10812|        1|SOVNKJI12A8C13CB0D|Take It To Da Hou...|2001|
+------------------+---------------+---------------+------

In [17]:
# get filepath to log data file
log_data = "s3a://udacity-dend/log_data/*/*"

# read log data file
log_data_df = spark.read.json(log_data, mode='PERMISSIVE', columnNameOfCorruptRecord='corrupt_record')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
log_data_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [21]:
new_log = log_data_df.where(log_data_df.page=="NextSong")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# extract columns to create songs table
# song_id, title, artist_id, year, duration
songs_table = song_data_df.select("song_id", "title", "artist_id", "year", "duration").distinct()

# write songs table to parquet files partitioned by year and artist
songs_table.write.parquet('s3a://sparkify-emr-output/'+'songs_table', mode="overwrite" , partitionBy=['year', 'artist_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# extract columns to create artists table
# artist_id, name, location, latitude, longitude
artists_table = song_data_df.select([
    col("artist_id"),
    col("artist_name").alias("name"), 
    col("artist_location").alias("location"),
    col("artist_latitude").alias("latitude"), 
    col("artist_longitude").alias("longitude")
]).distinct()

# write artists table to parquet files 
artists_table.write.parquet('s3a://sparkify-emr-output/'+'artists_table', mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# extract columns for users table
# user_id, first_name, last_name, gender, level
users_table = new_log.select([
    col("userId").alias("user_id"),
    col("firstName").alias("first_name"),
    col("lastName").alias("last_name"),
    col("gender"),
    col("level")
]).distinct()

# write users table to parquet files
users_table.write.parquet('s3a://sparkify-emr-output/'+'users_table', mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# create datetime column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000).isoformat())

df = new_log.withColumn("start_time", get_timestamp("ts").cast("timestamp"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
# get_datetime =  udf(lambda x: datetime.to_date(x))

# df = df.withColumn("start_time", get_datetime("timestamp").cast("timestamp"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
# extract columns to create time table
# start_time, hour, day, week, month, year, weekday
time_table = df.select("start_time").dropDuplicates()\
.withColumn("hour", hour(col("start_time"))).withColumn("day", dayofmonth(col("start_time")))\
.withColumn("week", weekofyear(col("start_time"))).withColumn("month", month(col("start_time")))\
.withColumn("year", year(col("start_time"))).withColumn("weekday", dayofweek(col("start_time")))

# write time table to parquet file partioned by year and month
time_table.write.parquet('s3a://sparkify-emr-output/'+'time_table', mode="overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# read in song data to use for songplays table
song_df = songs_table
song_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)

In [46]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)

In [58]:
# extract columns from joined song and log datasets to create songplays table
# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
songplays_table = df.join(song_data_df, df.artist==song_data_df.artist_name, 'inner')\
                  .withColumn("songplay_id", monotonically_increasing_id())\
                  .withColumn("year", year(df.start_time))\
                  .withColumn("month", month(col("start_time")))\

songplays_table = songplays_table.select(
                          [col("songplay_id"),
                          col("start_time"), 
                          col("userId").alias("user_id"), 
                          col("level"), 
                          col("song_id"),
                          col("artist_id"),
                          col("sessionId").alias("session_id"),
                          col("location"),
                          col("userAgent").alias("user_agent"),
                          col("year"),
                          col("month")
                          ])
            

# write songplays table to parquet files partitioned by year and month
# .write.parquet('s3a://sparkify-emr-output/'+'songplays_table', mode="overwrite" , partitionBy=['year', 'month'])
songplays_table.write.parquet('s3a://sparkify-emr-output/'+'songplays_table', mode="overwrite" , partitionBy=['year', 'month'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…